## Project NLP | Business Case: Automated Customer Reviews
Review classification file. Using Distilbert base uncased (dbu)

## Downloads

In [1]:
# The following downloads are needed to run this notebook
# !pip -q install transformers datasets
!pip -q install -U "transformers>=4.30" "accelerate>=0.21" datasets
import transformers, torch
print("transformers:", transformers.__version__, "| torch:", torch.__version__)

transformers: 4.55.4 | torch: 2.8.0+cu126


## Libraries

In [ ]:
# The following imports should be done to run this notebook
import os, json, importlib
import pandas as pd
import numpy as np
import torch

from datetime import datetime
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
)

import transformers
import transformers.training_args as ta
importlib.reload(transformers)
importlib.reload(ta)

from transformers.training_args import TrainingArguments
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback, 
)
from datasets import Dataset

os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("transformers:", transformers.__version__)
print("`evaluation_strategy` in TrainingArguments? ",
      "evaluation_strategy" in TrainingArguments.__init__.__code__.co_varnames)


transformers: 4.55.4
`evaluation_strategy` in TrainingArguments?  False


## Loading preprocessed data

In [3]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths
BASE = "/content/drive/MyDrive/Project_NLP"
PATH_FULL   = os.path.join(BASE, "video_games_preprocessed.parquet")
PATH_SAMPLE = os.path.join(BASE, "video_games_preprocessed_sample1pct.parquet")
PATH_CFG    = os.path.join(BASE, "preprocess_config.json")

# Loading
df        = pd.read_parquet(PATH_FULL)
df_sample = pd.read_parquet(PATH_SAMPLE)
cfg       = json.load(open(PATH_CFG))

df_use = df_sample.copy()

# Checking
print(" - sample: ", len(df_sample), "rows")
print(" - config keys:", list(cfg.keys()))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 - sample:  46246 rows
 - config keys: ['text_col', 'label_col', 'label2id', 'max_length', 'class_counts', 'class_weights']


# Splitting data set


In [ ]:
# Just inspecting + record class stats
label_col = cfg["label_col"]
order = ["negative", "neutral", "positive"]

def class_stats(series, order):
    counts = series.value_counts().reindex(order, fill_value=0)
    weights = (counts.sum() / (len(order) * counts)).astype(float)  # inverse freq
    return counts, weights

counts_use, weights_use = class_stats(df_use[label_col], order)

print("WORKING DF counts:\n", counts_use.to_dict())
print("WORKING DF class_weights:\n", weights_use.to_dict())

WORKING DF counts:
 {'negative': 8394, 'neutral': 3401, 'positive': 34451}
WORKING DF class_weights:
 {'negative': 1.8364704947978714, 'neutral': 4.532588454376164, 'positive': 0.4474567743558484}


In [5]:
# train/val/test split on the sample dataset (70/15/15)
text_col  = cfg["text_col"]
label_col = cfg["label_col"]
order     = ["negative", "neutral", "positive"]

train_df, tmp_df = train_test_split(
    df_use, test_size=0.30, stratify=df_use[label_col], random_state=42
)
val_df, test_df = train_test_split(
    tmp_df, test_size=0.50, stratify=tmp_df[label_col], random_state=42
)

def pct(d):
    c = d[label_col].value_counts().reindex(order, fill_value=0)
    return (c / c.sum() * 100).round(2)

print("Sizes -> train/val/test:", len(train_df), len(val_df), len(test_df))
print("\nClass % in train:\n", pct(train_df))
print("\nClass % in val:\n",   pct(val_df))
print("\nClass % in test:\n",  pct(test_df))

Sizes -> train/val/test: 32372 6937 6937

Class % in train:
 sentiment
negative    18.15
neutral      7.36
positive    74.49
Name: count, dtype: float64

Class % in val:
 sentiment
negative    18.15
neutral      7.35
positive    74.50
Name: count, dtype: float64

Class % in test:
 sentiment
negative    18.15
neutral      7.35
positive    74.50
Name: count, dtype: float64


As seen in preprocessing, the data set was very positive balanced. So we should rebalance the training split before we can do model training.

Negative 18.15%, neutral 7.35%, positive 74.50%

## Tokenizing for Distilbert-base-uncased

In [ ]:
# Setting up config for model distilbert-base-uncased
MODEL_NAME_dbu = "distilbert-base-uncased"
SHORT_dbu      = "dbu"                     # <- shortname for this model
MAX_LENGTH     = cfg["max_length"]
label2id       = cfg["label2id"]
text_col       = cfg["text_col"]
label_col      = cfg["label_col"]
order          = ["negative","neutral","positive"]

# recompute class weights from the TRAIN split (recommended)
train_counts_dbu = train_df[label_col].value_counts().reindex(order, fill_value=0)
train_class_weights_dbu = (train_counts_dbu.sum() / (len(order) * train_counts_dbu)).astype(float).to_dict()
print("DBU train counts:", train_counts_dbu.to_dict())
print("DBU train class weights:", train_class_weights_dbu)


DBU train counts: {'negative': 5876, 'neutral': 2381, 'positive': 24115}
DBU train class weights: {'negative': 1.8363966417063762, 'neutral': 4.531989360212796, 'positive': 0.4474669984103946}


In [7]:
# Making data frames to keep only text and labels/sentiment
def prep_for_model(df_):
    d = df_[[text_col, label_col]].copy()
    d["labels"] = d[label_col].map(label2id).astype("int64")
    d.rename(columns={text_col: "text"}, inplace=True)
    return d[["text","labels"]]

train_p_dbu = prep_for_model(train_df)
val_p_dbu   = prep_for_model(val_df)
test_p_dbu  = prep_for_model(test_df)

for name, d in [("train_dbu", train_p_dbu), ("val_dbu", val_p_dbu), ("test_dbu", test_p_dbu)]:
    print(name, d.shape)

train_dbu (32372, 2)
val_dbu (6937, 2)
test_dbu (6937, 2)


In [8]:
# Converting to datasets, should be more memory effiecient
ds_train_dbu = Dataset.from_pandas(train_p_dbu, preserve_index=False)
ds_val_dbu   = Dataset.from_pandas(val_p_dbu,   preserve_index=False)
ds_test_dbu  = Dataset.from_pandas(test_p_dbu,  preserve_index=False)

print(ds_train_dbu, ds_val_dbu, ds_test_dbu)

Dataset({
    features: ['text', 'labels'],
    num_rows: 32372
}) Dataset({
    features: ['text', 'labels'],
    num_rows: 6937
}) Dataset({
    features: ['text', 'labels'],
    num_rows: 6937
})


In [ ]:
# Load tokenizer and tokenize, see note if ram issues
tokenizer_dbu = AutoTokenizer.from_pretrained(MODEL_NAME_dbu)
def tok_fn_dbu(batch):
    return tokenizer_dbu(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,                 # dynamic padding via collator
        return_attention_mask=True,
    )

# NOTE: Tokenizing the FULL dataset is heavy in Colab.
# If there are RAM/time issues, try first with a subset by uncommenting:
# ds_train_dbu = ds_train.select(range(200_000))  # e.g., first 200k for a dry run

ds_train_tok_dbu = ds_train_dbu.map(tok_fn_dbu, batched=True, remove_columns=["text"])
ds_val_tok_dbu   = ds_val_dbu.map(tok_fn_dbu,     batched=True, remove_columns=["text"])
ds_test_tok_dbu  = ds_test_dbu.map(tok_fn_dbu,    batched=True, remove_columns=["text"])

print("Tokenized (dbu):", len(ds_train_tok_dbu), len(ds_val_tok_dbu), len(ds_test_tok_dbu))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/32372 [00:00<?, ? examples/s]

Map:   0%|          | 0/6937 [00:00<?, ? examples/s]

Map:   0%|          | 0/6937 [00:00<?, ? examples/s]

Tokenized (dbu): 32372 6937 6937


In [10]:
# Setting PyTorch fromat
ds_train_tok_dbu.set_format(type="torch")
ds_val_tok_dbu.set_format(type="torch")
ds_test_tok_dbu.set_format(type="torch")

data_collator_dbu = DataCollatorWithPadding(tokenizer=tokenizer_dbu, padding="longest")

ex = ds_train_tok_dbu[0]
print("DBU example keys:", ex.keys(), "| input_ids length:", len(ex["input_ids"]))

DBU example keys: dict_keys(['labels', 'input_ids', 'attention_mask']) | input_ids length: 3


## Model training with distilbert base uncased

In [11]:
# Label maps
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Model (DistilBERT baseline)
model_dbu = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME_dbu,
    num_labels=3,
    id2label=id2label,
    label2id=label2id,
)

def compute_metrics_dbu(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0
    )
    return {"accuracy": acc, "f1_macro": f1_macro, "precision_macro": p_macro, "recall_macro": r_macro}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Class weighted training to handle that the training test is imbalanced
label_order = ["negative", "neutral", "positive"]

# copy current class weights in the train split
w = {k: train_class_weights_dbu[k] for k in label_order}

# boost neutral
w["neutral"] *= 1.3

# checking weight used
weight_tensor_dbu = torch.tensor([w[k] for k in label_order], dtype=torch.float32)
print("class weights used (neg, neu, pos):", [float(x) for x in weight_tensor_dbu])

class WeightedTrainerDBU(Trainer):
    # accept extra kwargs from new Trainer (e.g., num_items_in_batch) to stay future-proof
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        # make sure labels are correct dtype for CE loss
        if labels.dtype != torch.long:
            labels = labels.long()

        outputs = model(**inputs)
        logits = outputs.logits

        loss_fct = torch.nn.CrossEntropyLoss(
            weight=weight_tensor_dbu.to(logits.device)
        )
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

class weights used (neg, neu, pos): [1.8363966941833496, 5.8915863037109375, 0.44746699929237366]


In [ ]:
# Setting training arguments
import importlib, transformers
importlib.reload(transformers)
from transformers.training_args import TrainingArguments

def make_training_args_dbu(output_dir=f"runs_{SHORT_dbu}"):
    """
    Build TrainingArguments with a compatibility shim:
    - Attempt A: modern API (>=3.x): eval/save 'epoch' + best model at end
    - Attempt B: fallback (very old API): no eval during training,
      no 'best model at end' (avoids the mismatch error).
    """
    # ---- Base hyperparams (tuned for the SAMPLE split) ----
    base = dict(
        output_dir=output_dir,

        # Optimization
        learning_rate=1e-5,
        weight_decay=0.01,

        # Batching (effective train batch ≈ 16 * 2 = 32)
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=2,

        # Train length on SAMPLE
        num_train_epochs=3,

        # Logging
        logging_strategy="steps",
        logging_steps=100,

        # Mixed precision
        fp16=True,

        # Keep disk tidy & avoid external loggers by default
        save_total_limit=2,
        report_to="none",
    )

    # ---- Attempt A: modern API (eval/save must MATCH) ----
    try:
        args = TrainingArguments(
            **base,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1_macro",
            greater_is_better=True,
        )
        print("[TrainingArguments] Using modern API (eval/save='epoch').")
        return args
    except TypeError as e:
        print("[TrainingArguments] Modern API not supported:", e)

    # ---- Attempt B: minimal fallback (NO eval during training) ----
    # Remove modern-only fields and DISABLE 'best model at end'
    base_min = dict(base)
    base_min.pop("logging_strategy", None)   # some very old builds don't have this
    args = TrainingArguments(
        **base_min,
        # No evaluation during training in this fallback
        load_best_model_at_end=False,  # <- avoids the mismatch error
        # Don't set evaluation_strategy/save_strategy here
    )
    print("[TrainingArguments] Using minimal fallback (no eval during training; no best model at end).")
    return args

# Build args and quick sanity check
args_dbu = make_training_args_dbu(f"runs_{SHORT_dbu}")
print("Has 'evaluation_strategy':", hasattr(args_dbu, "evaluation_strategy"))
print("Has 'load_best_model_at_end':", getattr(args_dbu, "load_best_model_at_end", None))


[TrainingArguments] Modern API not supported: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'
[TrainingArguments] Using minimal fallback (no eval during training; no best model at end).
Has 'evaluation_strategy': False
Has 'load_best_model_at_end': False


In [ ]:
# Training
trainer_dbu = WeightedTrainerDBU(
    model=model_dbu,
    args=args_dbu,                       # from builder
    train_dataset=ds_train_tok_dbu,
    eval_dataset=ds_val_tok_dbu,         # optional; used when you call evaluate()
    processing_class=tokenizer_dbu,      # replaces deprecated tokenizer=
    data_collator=data_collator_dbu,
    compute_metrics=compute_metrics_dbu,
)

train_result_dbu = trainer_dbu.train()


Step,Training Loss
100,1.035300
200,0.773300
300,0.692400
400,0.669100
500,0.633800
600,0.640900
700,0.628700
800,0.650000
900,0.601700
1000,0.608500


In [ ]:
# Validation metrics
val_metrics_dbu  = trainer_dbu.evaluate(ds_val_tok_dbu)
print("VAL (dbu):",  val_metrics_dbu)

test_metrics_dbu = trainer_dbu.evaluate(ds_test_tok_dbu, metric_key_prefix="test")
print("TEST (dbu):", test_metrics_dbu)

pred = trainer_dbu.predict(ds_test_tok_dbu)
y_true = pred.label_ids
y_pred = pred.predictions.argmax(axis=-1)

print(classification_report(y_true, y_pred,
                            target_names=["negative","neutral","positive"],
                            digits=3))
print("Confusion matrix (rows=true, cols=pred):\n",
      confusion_matrix(y_true, y_pred, labels=[0,1,2]))


VAL (dbu): {'eval_loss': 0.6241616606712341, 'eval_accuracy': 0.848205276055932, 'eval_f1_macro': 0.7060915324286569, 'eval_precision_macro': 0.689427548050409, 'eval_recall_macro': 0.7481854247628291, 'eval_runtime': 11.6154, 'eval_samples_per_second': 597.223, 'eval_steps_per_second': 18.682, 'epoch': 3.0}
TEST (dbu): {'test_loss': 0.6376691460609436, 'test_accuracy': 0.8435923309788093, 'test_f1_macro': 0.7043013038405533, 'test_precision_macro': 0.6869476072854783, 'test_recall_macro': 0.7523544787813262, 'test_runtime': 11.8366, 'test_samples_per_second': 586.064, 'test_steps_per_second': 18.333, 'epoch': 3.0}
              precision    recall  f1-score   support

    negative      0.793     0.809     0.801      1259
     neutral      0.294     0.569     0.388       510
    positive      0.973     0.879     0.924      5168

    accuracy                          0.844      6937
   macro avg      0.687     0.752     0.704      6937
weighted avg      0.891     0.844     0.862      69

## Saving model

In [16]:
# saving too google drive
BASE_SAVE = "/content/drive/MyDrive/Project_NLP/models"
RUN_TAG   = "dbu_sample_e3_lr1e-5_wNeutral1p3"
SAVE_DIR  = f"{BASE_SAVE}/{RUN_TAG}"

import os, json, datetime, numpy as np
os.makedirs(SAVE_DIR, exist_ok=True)
print("Saving to:", SAVE_DIR)


Saving to: /content/drive/MyDrive/Project_NLP/models/dbu_sample_e3_lr1e-5_wNeutral1p3


In [17]:
# Model + tokenizer
trainer_dbu.save_model(SAVE_DIR)
tokenizer_dbu.save_pretrained(SAVE_DIR)

('/content/drive/MyDrive/Project_NLP/models/dbu_sample_e3_lr1e-5_wNeutral1p3/tokenizer_config.json',
 '/content/drive/MyDrive/Project_NLP/models/dbu_sample_e3_lr1e-5_wNeutral1p3/special_tokens_map.json',
 '/content/drive/MyDrive/Project_NLP/models/dbu_sample_e3_lr1e-5_wNeutral1p3/vocab.txt',
 '/content/drive/MyDrive/Project_NLP/models/dbu_sample_e3_lr1e-5_wNeutral1p3/added_tokens.json',
 '/content/drive/MyDrive/Project_NLP/models/dbu_sample_e3_lr1e-5_wNeutral1p3/tokenizer.json')

In [ ]:
# Puting all diagnostics beside model, but in a separate "runs" area
RUNS_BASE = "/content/drive/MyDrive/Project_NLP/runs"
RUN_DIR    = f"{RUNS_BASE}/{RUN_TAG}"
os.makedirs(RUN_DIR, exist_ok=True)
print("Writing metrics & reports to:", RUN_DIR)

# 1) raw val/test metrics → JSON
with open(f"{RUN_DIR}/metrics.json", "w") as f:
    json.dump(
        {
            "val":  val_metrics_dbu,     # from trainer_dbu.evaluate(ds_val_tok_dbu)
            "test": test_metrics_dbu,    # from trainer_dbu.evaluate(ds_test_tok_dbu, metric_key_prefix="test")
        },
        f,
        indent=2
    )

# 2) classification report → CSV
label_order = ["negative", "neutral", "positive"]
rep_dict = classification_report(
    y_true, y_pred, target_names=label_order, output_dict=True, digits=3
)
pd.DataFrame(rep_dict).T.to_csv(f"{RUN_DIR}/classification_report.csv")

# 3) confusion matrix → CSV
cm = confusion_matrix(y_true, y_pred, labels=[0, 1, 2])
pd.DataFrame(
    cm,
    index=[f"true_{l}" for l in label_order],
    columns=[f"pred_{l}" for l in label_order],
).to_csv(f"{RUN_DIR}/confusion_matrix.csv")

# 4) test-set predictions → CSV
pd.DataFrame({"y_true": y_true, "y_pred": y_pred}).to_csv(
    f"{RUN_DIR}/preds_test.csv", index=False
)

# 5) compact run summary (to remember how it was trained) → JSON
summary = {
    "run_tag": RUN_TAG,
    "model_path": SAVE_DIR,
    "model_name": MODEL_NAME_dbu,            # e.g., "distilbert-base-uncased"
    "max_length": MAX_LENGTH,
    "epochs": getattr(args_dbu, "num_train_epochs", None),
    "learning_rate": getattr(args_dbu, "learning_rate", None),
    "train_sizes": {
        "train": len(ds_train_tok_dbu),
        "val":   len(ds_val_tok_dbu),
        "test":  len(ds_test_tok_dbu),
    },
    # if you used class weights:
    "class_weights_used": (
        [float(x) for x in weight_tensor_dbu.tolist()]
        if "weight_tensor_dbu" in globals() else None
    ),
    "notes": "DBU on SAMPLE; neutral boosted ×1.3",
}
with open(f"{RUN_DIR}/run_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("Saved: metrics.json, classification_report.csv, confusion_matrix.csv, preds_test.csv, run_summary.json")



Writing metrics & reports to: /content/drive/MyDrive/Project_NLP/runs/dbu_sample_e3_lr1e-5_wNeutral1p3
Saved: metrics.json, classification_report.csv, confusion_matrix.csv, preds_test.csv, run_summary.json
